### Quick mockup

In [1]:
import getpass
import os

open_ai_key = getpass.getpass("Open AI Key: ")

In [2]:
os.environ["OPENAI_API_KEY"] = open_ai_key

In [3]:
from langchain.llms import OpenAI 

# Create an LLM object
llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [4]:
llm("Hello, this is a test.")

"\n\nHello there! It's nice to meet you. What kind of test are you referring to?"

In [5]:
from IPython.display import Markdown

Markdown(llm("Hello, this is a test."))



Hello there! It's nice to meet you. What kind of test are you referring to?

In [6]:
Markdown(llm("What is zero-shot chain-of-thought prompting?"))



Zero-shot chain-of-thought prompting is a technique used to help people with cognitive impairments to generate ideas and thoughts. It involves prompting the person with a series of questions that are related to the topic at hand. The questions are designed to help the person to think of related ideas and to build a chain of thought. This technique can be used to help people with memory problems, language impairments, and other cognitive impairments to generate ideas and thoughts.

### Let's bring in some context

In [7]:
import arxiv

paper = next(arxiv.Search(id_list=["2205.11916"]).results()) #The next function returns the next item in an iterator

Markdown(paper.summary)

Pretrained large language models (LLMs) are widely used in many sub-fields of
natural language processing (NLP) and generally known as excellent few-shot
learners with task-specific exemplars. Notably, chain of thought (CoT)
prompting, a recent technique for eliciting complex multi-step reasoning
through step-by-step answer examples, achieved the state-of-the-art
performances in arithmetics and symbolic reasoning, difficult system-2 tasks
that do not follow the standard scaling laws for LLMs. While these successes
are often attributed to LLMs' ability for few-shot learning, we show that LLMs
are decent zero-shot reasoners by simply adding "Let's think step by step"
before each answer. Experimental results demonstrate that our Zero-shot-CoT,
using the same single prompt template, significantly outperforms zero-shot LLM
performances on diverse benchmark reasoning tasks including arithmetics
(MultiArith, GSM8K, AQUA-RAT, SVAMP), symbolic reasoning (Last Letter, Coin
Flip), and other logical reasoning tasks (Date Understanding, Tracking Shuffled
Objects), without any hand-crafted few-shot examples, e.g. increasing the
accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with
large InstructGPT model (text-davinci-002), as well as similar magnitudes of
improvements with another off-the-shelf large model, 540B parameter PaLM. The
versatility of this single prompt across very diverse reasoning tasks hints at
untapped and understudied fundamental zero-shot capabilities of LLMs,
suggesting high-level, multi-task broad cognitive capabilities may be extracted
by simple prompting. We hope our work not only serves as the minimal strongest
zero-shot baseline for the challenging reasoning benchmarks, but also
highlights the importance of carefully exploring and analyzing the enormous
zero-shot knowledge hidden inside LLMs before crafting finetuning datasets or
few-shot exemplars.

In [8]:
response = llm(f"""Here's a summary of the paper: {paper.summary})

Based on that summary, what is zero-shot chain-of-thought prompting?""")

Markdown(response)

 Zero-shot chain-of-thought prompting is a technique for eliciting complex multi-step reasoning through step-by-step answer examples without the need for hand-crafted few-shot examples. This technique uses a single prompt template that is versatile across very diverse reasoning tasks, and has been shown to significantly outperform zero-shot language model performances on benchmark reasoning tasks. This suggests that high-level, multi-task broad cognitive capabilities may be extracted by simple prompting.

### Make the paper searchable

In [9]:
import pypdf

In [10]:
paper_path = paper.download_pdf()

In [11]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(paper_path)
pages = loader.load_and_split()

In [12]:
len(pages)

49

In [13]:
content = "\n\n".join([page.page_content for page in pages[0:2]])

In [14]:
response = llm(f"""Here's the first two pages of a paper: {content})

Based on that content, what is zero-shot chain-of-thought prompting?""")

Markdown(response)



Zero-shot chain-of-thought prompting is a technique for eliciting complex multi-step reasoning through step-by-step answer examples without the need for any hand-crafted few-shot examples. It involves adding a prompt, such as "Let's think step by step," before each answer to facilitate step-by-step thinking. This technique has been shown to significantly outperform zero-shot language model performances on diverse benchmark reasoning tasks, including arithmetics, symbolic reasoning, and other logical reasoning tasks.

### Find relevant content using embedding search

In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)

In [22]:
docs = db.similarity_search("What is zero-shot chain-of-thought prompting?")

In [23]:
Markdown(docs[0].page_content)

language models like PaLM [Chowdhery et al., 2022]. The top row of Figure 1 shows standard
few-shot prompting against (few-shot) CoT prompting. Notably, few-shot learning was taken as a
given for tackling such difﬁcult tasks, and the zero-shot baseline performances were not even reported
in the original work [Wei et al., 2022]. To differentiate it from our method, we call Wei et al. [2022]
asFew-shot-CoT in this work.
3 Zero-shot Chain of Thought
We propose Zero-shot-CoT, a zero-shot template-based prompting for chain of thought reasoning.
It differs from the original chain of thought prompting [Wei et al., 2022] as it does not require
step-by-step few-shot examples, and it differs from most of the prior template prompting [Liu et al.,
2021b] as it is inherently task-agnostic and elicits multi-hop reasoning across a wide range of tasks
with a single template. The core idea of our method is simple, as described in Figure 1: add Let’s

### Use an existing chain

In [27]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [31]:
chain = load_qa_with_sources_chain(llm, chain_type="stuff")
query = "What is zero-shot chain-of-thought prompting?"
sources = db.similarity_search(query)
results = chain({"input_documents": sources, "question": query}, return_only_outputs=True)

In [32]:
Markdown(results["output_text"])

 Zero-shot chain-of-thought prompting is a single zero-shot prompt that elicits chain of thought from large language models across a variety of reasoning tasks, without requiring hand-crafting few-shot examples per task.
SOURCES: ./2205.11916v4.Large_Language_Models_are_Zero_Shot_Reasoners.pdf